# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [6]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [7]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [10]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

C:\users\19163\OneDrive\desktop\my_nlp_env\lib\site-packages\chromadb\api\collection_configuration.py:335: DeprecationWarning: legacy embedding function config: The CHROMA_OPENAI_API_KEY environment variable is not set.
  return json.dumps(create_collection_configuration_to_json(config))
C:\users\19163\OneDrive\desktop\my_nlp_env\lib\site-packages\chromadb\api\rust.py:236: DeprecationWarning: legacy embedding function config
  configuration=load_collection_configuration_from_json(
C:\users\19163\OneDrive\desktop\my_nlp_env\lib\site-packages\chromadb\api\models\CollectionCommon.py:158: DeprecationWarning: legacy embedding function config
  return load_collection_configuration_from_json(self._model.configuration_json)


### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [11]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 78afdc50-5d4f-4f21-8f9a-a1af34301ec6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4b287642-df02-4666-b881-613e1fc327d8
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4b287642-df02-4666-b881-613e1fc327d8
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress

# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


🤖 Agent: Senior Support Representative
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Support Quality Assurance Specialist
## Task: Review the response drafted by the Senior Support Representative for DeepLearningAI's inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered.


# Agent: Support Quality Assurance Specialist
## Final Answer: 
Dear DeepLearningAI team,

Thank you for reaching out to us with your inquiry regarding the process of adding memory to your Crew. We understand the significance of memory planning and the impact it can have on enhancing the performance of your AI agents. To effectively add memory to your Crew and maximize its potential, CrewAI offers a comprehensive framework that allows for advanced memory planning an

🚀 Crew: crew
├── 📋 Task: 4b287642-df02-4666-b881-613e1fc327d8
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: bbee0115-5520-437d-a01d-ca1663999171
       Status: Executing Task...
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 4b287642-df02-4666-b881-613e1fc327d8
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: bbee0115-5520-437d-a01d-ca1663999171
       Assigned to: Support Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bbee0115-5520-437d-a01d-ca1663999171                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 78afdc50-5d4f-4f21-8f9a-a1af34301ec6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the final result as Markdown.

In [13]:
# from IPython.display import Markdown
# Markdown(result)

from IPython.display import Markdown

# Extract the raw text or content from the CrewOutput object
text_result = str(result.raw) if hasattr(result, 'raw') else str(result)

# Display as Markdown
Markdown(text_result)

Dear DeepLearningAI team,

Thank you for reaching out to us with your inquiry regarding the process of adding memory to your Crew. We understand the significance of memory planning and the impact it can have on enhancing the performance of your AI agents. To effectively add memory to your Crew and maximize its potential, CrewAI offers a comprehensive framework that allows for advanced memory planning and collaboration capabilities. Below, we will address each aspect of your inquiry in detail:

1. **Utilizing Memory Planning**:
   By incorporating memory planning into your Crew structure, you provide your AI agents with the ability to retain vital information, context, and historical data. This, in turn, empowers your agents to make more informed decisions and improves their overall performance through enhanced memory recall.

2. **Implementing Intelligent Collaboration**:
   Enabling intelligent collaboration within your Crew allows for the seamless sharing of insights and efficient task coordination. By fostering a collaborative environment, the shared memory among agents contributes significantly to achieving complex objectives with greater ease.

3. **Utilizing Role-Based Agents and Flexible Tools**:
   Defining specific roles for your AI agents, such as researchers or analysts, and equipping them with custom tools and APIs ensures that each agent can leverage their expertise effectively. This approach, coupled with memory-enhanced capabilities, enhances the interactions with external services and optimizes task performance within the Crew.

4. **Task Management and Sequential Workflows**:
   Leveraging CrewAI's task management features enables you to define sequential workflows, streamline task dependencies, and ensure efficient memory utilization by agents. This structured approach enhances overall productivity and facilitates the successful completion of objectives within your Crew.

5. **Combining Crews with Flows for Enhanced Autonomy**:
   Consider combining Crews designed for collaborative intelligence with Flows, offering structured automations. This strategic combination allows for balancing autonomy levels with precise memory-infused workflow execution, leading to improved outcomes in your AI initiatives.

In conclusion, by integrating memory planning, fostering intelligent collaboration, utilizing role-based agents with flexible tools, managing tasks efficiently, and combining Crews with Flows, you can effectively add memory to your Crew and drive successful outcomes in your AI endeavors.

Should you require further assistance or have additional queries, please feel free to reach out. We are here to support you in optimizing your AI operations and enhancing the capabilities of your Crew.

Best regards,

Senior Support Representative
CrewAI Support Team